In [1]:
import pandas as pd

In [4]:
sales = pd.read_csv("../DataSet/start_vgsales.csv")
steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]
steam.columns = ["player_id", "game_title", "type", "value"]

C:\Users\david\AppData\Local\Temp\ipykernel_12812\560633259.py:2: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]


In [5]:
steam = steam[steam["type"] == "purchase"]

In [6]:
df = steam.merge(sales, how='inner',left_on="game_title", right_on="Name")
df.head()

,player_id,game_title,type,value,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,151603712,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
1,151603712,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09
2,151603712,Fallout 4,purchase,1.0,1632,Fallout 4,PC,2015.0,Role-Playing,Bethesda Softworks,0.50,0.63,0.00,0.10,1.23
3,87445402,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
4,87445402,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09


In [7]:
df2 = df[["player_id", "Genre", "value"]]
df3 = df2.groupby(["player_id", "Genre"]).count()

In [8]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np

In [14]:
users = df3.reset_index()
users["player_id"] = np.asarray(users["player_id"]).astype(np.int32)
users

,player_id,Genre,value
0,86540,Action,7
1,86540,Adventure,2
2,86540,Racing,5
3,86540,Shooter,25
4,86540,Strategy,4
...,...,...,...
8953,99723205,Shooter,12
8954,99723205,Strategy,1
8955,99766416,Shooter,2
8956,99812428,Strategy,2


In [15]:
genres = users["Genre"].unique()
#genres = pd.DataFrame(data=genres, columns=['Genre'])
genres

array(['Action', 'Adventure', 'Racing', 'Shooter', 'Strategy',
       'Role-Playing', 'Simulation', 'Sports', 'Fighting', 'Misc',
       'Puzzle', 'Platform'], dtype=object)

In [18]:
tf.random.set_seed(42)
users_tensor = tf.data.Dataset.from_tensor_slices(dict(users))
users_shuffled = users_tensor.shuffle(seed=42, buffer_size=100_000, reshuffle_each_iteration=False)

users_train = users_shuffled.take(8_000)
users_test = users_shuffled.skip(8_000).take(8958)

In [19]:
genres_tensor = tf.data.Dataset.from_tensor_slices(genres)

In [20]:
for r in genres_tensor.take(5):
    print(r)

tf.Tensor(b'Action', shape=(), dtype=string)
tf.Tensor(b'Adventure', shape=(), dtype=string)
tf.Tensor(b'Racing', shape=(), dtype=string)
tf.Tensor(b'Shooter', shape=(), dtype=string)
tf.Tensor(b'Strategy', shape=(), dtype=string)


In [21]:
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(users_tensor.map(lambda x: x["player_id"]))

genres_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
genres_vocabulary.adapt(genres_tensor)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [22]:
from typing import Dict, Text
class MovieLensModel(tfrs.Model):
     # We derive from a custom base class to help reduce boilerplate. Under the hood,
     # these are still plain Keras Models.

    def __init__(
            self,
            user_model: tf.keras.Model,
            genre_model: tf.keras.Model,
            task: tfrs.tasks.Retrieval):
        super().__init__()

          # Set up user and movie representations.
        self.user_model = user_model
        self.genre_model = genre_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["player_id"])
        genre_embeddings = self.genre_model(features["Genre"])

        return self.task(user_embeddings, genre_embeddings)

In [23]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 10)
])
genre_model = tf.keras.Sequential([
    genres_vocabulary,
    tf.keras.layers.Embedding(genres_vocabulary.vocabulary_size(), 10)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    genres_tensor.batch(128).map(genre_model)
))

In [24]:
model = MovieLensModel(user_model, genre_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(users_train.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    genres_tensor.batch(100).map(lambda title: (title, model.genre_model(title))))

# Get some recommendations.
_, titles = index(np.array([2753525]))
print(f"Top 3 recommendations for user 2753525: {titles[0, :10]}")
print(f"Scores: {_[0, :10]}")


Epoch 1/3
2/2 [==============================] - 3s 197ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0344 - factorized_top_k/top_5_categorical_accuracy: 0.3530 - factorized_top_k/top_10_categorical_accuracy: 0.8405 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 32873.3945 - regularization_loss: 0.0000e+00 - total_loss: 32873.3945
Epoch 2/3
2/2 [==============================] - 0s 177ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0279 - factorized_top_k/top_5_categorical_accuracy: 0.2457 - factorized_top_k/top_10_categorical_accuracy: 0.5266 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 34221.5742 - regularization_loss: 0.0000e+00 - total_loss: 34221.5742
Epoch 3/3
2/2 [==============================] - 0s 221ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1119 - factorized_top_k/top_5_categorical_accuracy: 0.6701

In [25]:
model.evaluate(users_test, return_dict=True)

ValueError: in user code:

    File "c:\Users\david\miniconda3\envs\machine_learning\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\david\miniconda3\envs\machine_learning\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\david\miniconda3\envs\machine_learning\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\david\miniconda3\envs\machine_learning\lib\site-packages\tensorflow_recommenders\models\base.py", line 88, in test_step
        loss = self.compute_loss(inputs, training=False)
    File "C:\Users\david\AppData\Local\Temp\ipykernel_12812\2414308280.py", line 26, in compute_loss
        return self.task(user_embeddings, genre_embeddings)
    File "c:\Users\david\miniconda3\envs\machine_learning\li